In [22]:
import astropy.io.fits as fits
from pathlib import Path
import glob, os
import matplotlib.pyplot as plt
import numpy as np
from astropy.stats import median_absolute_deviation as mad

def rescale(image, mref, sref, roi=None):
    imageV = image if roi is None else image[roi]
    m1 = np.nanmedian(imageV)
#     s1 = np.sqrt(bwmv(imageV))
    s1 = mad(imageV, ignore_nan=True)
    ki = sref/s1
    rimage = ki*(image - m1) + mref
    return rimage

In [5]:
%matplotlib widget

In [54]:
datadir = Path("C:\Data/FIRECAPTURE/Jup/221220/pipp_20201224_183035/Jup_175008")
datadir.exists()

True

In [58]:
rfits = sorted(list(datadir.glob('r_Jup*.fit')))
gfits = sorted(list(datadir.glob('g_Jup*.fit')))
len(rfits), len(gfits)

(2343, 2343)

In [59]:
rsample = fits.getdata(rfits[0])
gsample = fits.getdata(gfits[0])

In [60]:
plt.figure(figsize=(5,5))
plt.imshow(rsample, cmap='gray', origin='lower')
plt.axis('off')
plt.figure(figsize=(5,5))
plt.imshow(gsample, cmap='gray', origin='lower')
plt.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.5, 282.5, -0.5, 268.5)

In [61]:
gdata = np.array([fits.getdata(f) for f in gfits])
gdata.shape

(2343, 269, 283)

In [63]:
gdata2 = gdata.astype(np.float32).copy()
gdata2[gdata2 == 0] = np.nan

In [64]:
med_gdata = np.median(gdata, axis=0)
mean_gdata = gdata.mean(axis=0)

In [65]:
mref = np.nanmedian(gdata2)
sref = np.nanstd(gdata2)

In [66]:
gdata3 = np.array([rescale(data, mref, sref) for data in gdata2])
gdata3[np.isnan(gdata3)] = 0

In [67]:
med_gdata2 = np.median(gdata3, axis=0)
mean_gdata2 = np.median(gdata3, axis=0)

In [68]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize= (10,6))
axs[0, 0].imshow(gdata[0], origin='lower', cmap='gray')
axs[0, 0].set_title('raw')
axs[0, 0].axis('off')
axs[0, 1].imshow(mean_gdata, origin='lower', cmap='gray')
axs[0, 1].set_title('mean')
axs[0, 1].axis('off')
axs[0, 2].imshow(med_gdata, origin='lower', cmap='gray')
axs[0, 2].set_title('median')
axs[0, 2].axis('off')

axs[1, 0].imshow(gdata3[0], origin='lower', cmap='gray')
axs[1, 0].set_title('raw')
axs[1, 0].axis('off')
axs[1, 1].imshow(mean_gdata2, origin='lower', cmap='gray')
axs[1, 1].set_title('mean rescaled')
axs[1, 1].axis('off')
axs[1, 2].imshow(med_gdata2, origin='lower', cmap='gray')
axs[1, 2].set_title('median rescaled')
axs[1, 2].axis('off')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
med_gdata2.max()

122.65754

In [78]:
med_gdata_tiff = np.round(med_gdata2/255 * 65535).astype(np.uint16)

In [79]:
from imageio import imwrite

In [80]:
imwrite('C:\Data/FIRECAPTURE/Jup/221220/green_jupiter.tiff', np.flipud(med_gdata_tiff))